对各招聘信息的企业人才需求画像进行了建模，该模型能通过求职者的基本信息（渴望最低工资、最高工资、学历、工作经验、工作地点），生成可求职企业的基本画像（公司性质与公司规模）。该功能可以帮助各招聘门户网站完善其求职搜索功能，在用户进行搜索后迅速缩小适合企业的范围，进行更加迅速有效的职位推荐。

In [65]:
import pandas as pd
import os
import numpy as np
from math import cos, sin, atan2, sqrt, pi, radians, degrees, asin
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, MultiLabelBinarizer, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error, accuracy_score, precision_score
from sklearn.model_selection import train_test_split

该问题的本质是多标签分类问题。Multi-Label Machine Learning(MLL算法)是指预测模型中存在多个y值，具体分为两类不同情况：  
- 多个预测y值  
- 在分类模型中，一个样例可能存在多个不固定的类别  

在本项目中，通过输入用户信息，预测公司的两个标签，为`公司性质`与`公司规模`。  

根据多标签问题的复杂性，可以将问题分为两大类：  
- 待预测值之间存在相互的依赖关系
- 待预测值之间不存在依赖关系  

多标签问题的学习策略主要有三种，分别是一阶策略，二阶策略和高阶策略。  
1. 一阶策略不考虑标签相关性，效率高；
2. 二阶策略考虑两个标签之间的相关性；
3. 高阶策略考虑多个标签之间的相关性，性能好。  

目前有很多关于多标签的学习算法，依据解决问题的角度，这些算法可以分为两大类:一是`问题转换策略`，二是`算法适应性策略`。  
`问题转换策略`是转化问题数据，使之使用现有算法,是一种将多标签的分类问题转换成为单标签模型构造的问题，然后将模型合并的一种方式。可分为：  
- Binary Relevance(二元关联)：标签之间无关联  
- Classifier Chains(分类器链)：标签之间有依赖关系  
- Calibrated Label Ranking(LP法)：两两标签之间有关系  

`算法适应性策略`是指针对某一特定的算法进行扩展，从而能够处理多标记数据，改进算法，适用数据。主要有：  
- Multi Label-KNN



### 数据获取

读取数据文件，抽取出特征和标签，并对其进行独热编码，按`4:1`的比例进行训练集和测试集的划分。

In [2]:
df = pd.read_csv('../data/info-final.csv')
df.columns

Index(['c_name', 'c_nature', 'c_scale', 'w_place', 'w_field', 'w_experience',
       'education', 's_min', 's_max', 'vacancies'],
      dtype='object')

In [3]:
def sample_data(df_unsampled):
    df_other = df_unsampled.drop(df[df['c_nature'] == '民营'].index, axis = 0)
    df0 = df_unsampled.groupby(['c_nature','c_scale']).get_group(('民营','20人以下'))
    df1 = df_unsampled.groupby(['c_nature','c_scale']).get_group(('民营','20-99人')).sample(n=500)
    df2 = df_unsampled.groupby(['c_nature','c_scale']).get_group(('民营','100-499人')).sample(n=500)
    df3 = df_unsampled.groupby(['c_nature','c_scale']).get_group(('民营','500-999人')).sample(n=500)
    df4 = df_unsampled.groupby(['c_nature','c_scale']).get_group(('民营','1000-9999人')).sample(n=500)
    df5 = df_unsampled.groupby(['c_nature','c_scale']).get_group(('民营','10000人以上'))
    df_sampled = pd.concat([df_other, df0, df1, df2, df3, df4, df5], axis = 0)
    return df_sampled

In [4]:
company_info = pd.DataFrame(df, columns = ['c_nature','c_scale'])
features = df.drop(['c_name', 'c_nature', 'c_scale', 'w_field', 'vacancies'], axis = 1)

In [5]:
features = np.array(pd.get_dummies(features))
company_info = np.array(pd.get_dummies(company_info))

In [ ]:
# le = LabelEncoder()
# features.w_place = le.fit_transform(features.w_place)
# features.w_experience = le.fit_transform(features.w_experience)
# features.education = le.fit_transform(features.education)
# features.s_min = le.fit_transform(features.s_min)
# features.s_max = le.fit_transform(features.s_max)
# company_info.c_nature = le.fit_transform(company_info.c_nature)
# company_info.c_scale = le.fit_transform(company_info.c_scale)
# ohe = OneHotEncoder()
# features = ohe.fit_transform(features)
# company_info = ohe.fit_transform(company_info)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()
# features = mlb.fit_transform(features.values)
# company_info = mlb.fit_transform(company_info.values)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, company_info, test_size=0.2, random_state=2)

## Problem Transformation Methods (转换策略)
### Binary Relevance（二元关联）

Binary Relevance的核心思想是将多标签分类问题进行分解，将其转换为q个二元分类问题，其中每个二元分类器对应一个待预测的标签。  
由于没有考虑标签之间的相关性，是**一阶策略**。  
优点： 
- 估计单标签分类器；
- 可以推广到超出标签组合的范围；
- 实现方式简单，容易理解；
- 当y值之间不存在相关的依赖关系的时候，模型的效果不错  

缺点：  
- 标签数目很多的时候不适合；
- 忽略标签之间的相关性；
- 如果y直接存在相互的依赖关系，那么最终构建的模型的泛化能力比较弱；
- 需要构建q个二分类器，q为待预测的y值数量，当q比较大的时候，需要构建的模型会比较多

构造二分类器的方法使用one-vs-rest的方式。可以直接使用如下接口实现，其中的基分类器可以使用任意sklearn中的预设分类器。  
使用scikit-multilearn工具包进行策略及模型的构建与训练。

In [17]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

使用Decision Tree决策树分类器：

In [67]:
# Decision Tree
classifier = BinaryRelevance(DecisionTreeClassifier())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.7305110033569564

使用Naive Bayes朴素贝叶斯模型：

In [68]:
# Naive Bayes
classifier = BinaryRelevance(GaussianNB())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.16797928008879962

使用SVM分类器：

In [87]:
# SVM
classifier = BinaryRelevance(SVC())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.8103486077614558

使用Random Forest Classifier随即森林分类器：

In [70]:
# Random Forest Classifier
classifier = BinaryRelevance(RandomForestClassifier())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.7370261317629738

### Classifier Chains (分类器链)

Classifier Chains的核心思想是将多标签分类问题进行分解，将其转换成为一个二元分类器链的形式，其中链后的二元分类器的构建式在前面分类器预测结果的基础上的。在模型构建的时候，首先将标签顺序进行shuffle打乱排序操作，然后按照从头到尾分别构建每个标签对应的模型。  
虽然还是作为二分类问题解决的，但以链式的方式随机考虑了多个标签的相关性，这是**高阶策略**。  
优点：  
- 实现方式相对比较简单，容易理解；  
- 考虑标签之间的依赖关系，最终模型的泛化能力相对于Binary Relevance方式构建的模型效果要好。  

缺点：  
- 很难找到一个比较适合的标签依赖关系。

In [71]:
from skmultilearn.problem_transform import ClassifierChain

In [72]:
# Decision Tree
classifier = ClassifierChain(DecisionTreeClassifier())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.5928023153391216

In [73]:
# Naive Bayes
classifier = ClassifierChain(GaussianNB())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.4583333333333333

In [88]:
# SVM
classifier = ClassifierChain(SVC())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.601963746223565

In [74]:
# Random Forest Classifier
classifier = ClassifierChain(RandomForestClassifier())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.6071248741188319

### Label Powerset (LP法)

Label Powerset的核心思想是将多标签学习转化为一个多类分类问题，每个不同的标签组合认为是一个不同的类。  
考虑了多个标签之间的相关性，是**高阶策略**。  

优点：
- 利用一个分类器考虑了多标签的相关性；
- 在训练数据包括全部标签组合时通常是最优解。  

缺点
- 要求训练数据包括所有的标签组合；
- 当标签空间大时，很容易过拟合。

In [75]:
from skmultilearn.problem_transform import LabelPowerset

In [76]:
# Decision Tree
classifier = LabelPowerset(DecisionTreeClassifier())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.6125377643504532

In [77]:
# Naive Bayes
classifier = LabelPowerset(GaussianNB())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.3588872104733132

In [89]:
# SVM
classifier = LabelPowerset(SVC())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.62865055387714

In [78]:
# Random Forest Classifier
classifier = LabelPowerset(RandomForestClassifier())
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

0.6098942598187311

## Algorithm Adaptation (算法适应性策略)
### MLKNN

MLKNN的核心思想是对于每一个实例而言，先获取距离它最近的K个实例，然后使用这些实例的标签集合，通过最大后验概率(MAP)来判断这个实例的预测标签集合的值。

In [79]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

In [86]:
# MLKNN
classifier = MLkNN(k=10)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred, average='micro')

/anaconda3/envs/bigdata/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.7525525525525526

In [ ]:
# GridSearchCV
parameters = {'k': range(1,10), 's': [0.5, 0.7, 1.0]}
score = 'accuracy'

clf = GridSearchCV(MLkNN(), parameters, scoring=score)
clf.fit(X_train, y_train)
print (clf.best_params_, clf.best_score_)
y_pred = clf.predict(X_test)

precision_score(y_test, y_pred, average='micro')

### Neural Network

由于传统的基于统计的方法最终准确率结果不佳，我们采用MLP神经网络模型进行训练和预测。  

In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Activation
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support

In [84]:
n = X_train.shape[0]
dim_no = X_train.shape[1]
class_no = y_train.shape[1]
# create simple mlp
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(class_no, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
# train
model.fit(X_train, y_train, epochs=10, verbose=2, validation_data=(X_test,y_test))
# evaluation
y_pred = model.predict(X_test)
model.evaluate(X_test, y_test)

Train on 15886 samples, validate on 3972 samples
Epoch 1/10
15886/15886 - 1s - loss: 0.3776 - accuracy: 0.8953 - val_loss: 0.2298 - val_accuracy: 0.9248
Epoch 2/10
15886/15886 - 1s - loss: 0.2107 - accuracy: 0.9252 - val_loss: 0.2000 - val_accuracy: 0.9252
Epoch 3/10
15886/15886 - 1s - loss: 0.1962 - accuracy: 0.9262 - val_loss: 0.1924 - val_accuracy: 0.9266
Epoch 4/10
15886/15886 - 1s - loss: 0.1906 - accuracy: 0.9278 - val_loss: 0.1881 - val_accuracy: 0.9277
Epoch 5/10
15886/15886 - 1s - loss: 0.1871 - accuracy: 0.9290 - val_loss: 0.1851 - val_accuracy: 0.9290
Epoch 6/10
15886/15886 - 1s - loss: 0.1844 - accuracy: 0.9295 - val_loss: 0.1827 - val_accuracy: 0.9293
Epoch 7/10
15886/15886 - 1s - loss: 0.1823 - accuracy: 0.9299 - val_loss: 0.1807 - val_accuracy: 0.9299
Epoch 8/10
15886/15886 - 1s - loss: 0.1805 - accuracy: 0.9303 - val_loss: 0.1790 - val_accuracy: 0.9302
Epoch 9/10
15886/15886 - 1s - loss: 0.1789 - accuracy: 0.9304 - val_loss: 0.1776 - val_accuracy: 0.9306
Epoch 10/10
158

[0.17632813691672722, 0.9304381]

## Results

为了获得统计上可信的结果，我们将实验重复20次并报告平均结果。  

|  模型   | 准确率  |  
|  :----  | ---- |
| **Binary Relevance** |
| DT | 0.731 |
| NB | 0.296 |
| SVM | 0.810 |
| RFC | 0.737 |
| **Classifier Chains** |
| DT | 0.593 |
| NB | 0.458 |
| SVM | 0.602 |
| RFC | 0.607 |
| **Label Powerset** |
| DT | 0.612 |
| NB | 0.358 |
| SVM | 0.628 |
| RFC | 0.610 |
| **Algorithm Adaptation** |
| MLKNN | 0.752 |
| **Neural Network** |
| MLP | 0.931 |

由以上结果，经过我们的分析，首先由于朴素贝叶斯模型给定输出类别的情况下,假设属性之间相互独立，这个假设在实际应用中往往是不成立的，在属性个数比较多或者属性之间相关性较大时，分类效果不好，所以无论何种转换策略，使用朴素贝叶斯模型分类的结果都不理想。